<a href="https://colab.research.google.com/github/krishnanlab/obnb/blob/main/tutorials/basic_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Tutorial for the Open Biomedical Network Benchmark package

## 1. Installation

Installation can be easily done via `pip`.

via PyPI (released or pre-release versions)
```bash
pip install obnb
```

or via GitHub (latest dev version)
```bash
pip install git+https://github.com/krishnanlab/obnb
```

In [ ]:
# Install latest dev version of OBNB
!pip install -q git+https://github.com/krishnanlab/obnb

Check if the package is installed successfully

In [ ]:
import obnb
print(f"Installed obnb {obnb.__version__}")

In [ ]:
import obnb.ext.pecanpy
print(f"Extension for PecanPy installed: {obnb.ext.pecanpy}")

## 2. Data downloading and processing

First, load the `obnb.data` module that contains "recipies" for processing
differentt selections of biological networks and gene annotation data.

We also need to specify (1) the path to which the data will be saved, and more
importantly, (2) the **version** of the data we want to retrieve. The version
option allows for flexible data retrieval (either retrieve data from source, or
retrieve from processed data archive) and also enable reproduction of the
downstream analysis.

In [ ]:
import obnb.data
import yaml

# Where do we want to save the data and related files to
root = "datasets"

# What version of the pre-processed data to download
data_version = "obnbdata-0.1.0"
# data_version = "latest"  # download data from source and process from scratch
# data_version = "current"  # use the latest archived data version

### 2.1. Biological networks

Let's start with an example of obtaining the `BioPlex` network, which is a
protein-protein interaction (PPI) network that is constructed via AP-MS on
human cell-lines ([HEK293T](https://www.synthego.com/hek293) and
[HCT116](https://imanislife.com/collections/cell-lines/hct116-cells/)).
Checkout other avaialble options for processed biomedical networks on the OBNB
benchmark
[README](https://github.com/krishnanlab/obnbench#data-stats-obnbdata-010-) page.

[1] Huttlin, Edward L., et al. "The BioPlex network: a systematic exploration of the human interactome." Cell 162.2 (2015): 425-440.

[2] Huttlin, Edward L., et al. "Dual proteome-scale networks reveal cell-specific remodeling of the human interactome." Cell 184.11 (2021): 3022-3040.

In [ ]:
# Download network from archive
g = obnb.data.BioPlex(root, version=data_version)

In [ ]:
# Once downloaded, it can be used in future acess without redownloading
g = obnb.data.BioPlex(root, version=data_version)

In [ ]:
# You can also force redownloading the data by specifying redownload=True
g = obnb.data.BioPlex(root, version=data_version, redownload=True)

You can also checkout more information about the processing done for this
network by looking into the config.

In [ ]:
print(yaml.dump(g.to_config()))

The gene IDs in the network can be accessed via the `node_ids` attribute, which
are [Entrez](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1761442/) gene ID by
default.

In [ ]:
print(f"The first gene in the network is {g.node_ids[0]!r}")
print(f"The second gene in the network is {g.node_ids[1]!r}")
print(f"The third gene in the network is {g.node_ids[2]!r}")

The graph `g` object is an instance of the `obnb.graph.SparseGraph` object.
But it could be easily converted into a dense adjacency matrix via `to_adjmat`

In [ ]:
adj = g.to_adjmat()
adj

### 2.2. Gene annotations

Setting up gene annotation tasks is a tedious process that include

1. Obtain annotations for gene-term associations and convert gene identifier to
   the desired option.
1. Obtain and construct ontology graph that represents the relationships among
   different terms.
1. Propagate the gene-term annotations upward the ontology graph.
1. Extract non-redundant representative gene sets (terms) from the propagated
   annotations.


Here, we use the [DisGeNET](https://www.disgenet.org/) disease gene annotations
with [MONDO](https://mondo.monarchinitiative.org/) disease ontology as an
example to set up the DisGeNET gene set collection.

[3] Piñero, Janet, et al. "DisGeNET: a comprehensive platform integrating information on human disease-associated genes and variants." Nucleic acids research (2016): gkw943.

[4] Vasilevsky, Nicole A., et al. "Mondo: Unifying diseases for the world, by the world." medRxiv (2022): 2022-04.

In [ ]:
# Download annotations and ontology from archive
gsc = obnb.data.DisGeNET(root, version=data_version)

In [ ]:
# Again, once downloaded and processed, it can be used in the future
gsc = obnb.data.DisGeNET(root, version=data_version)

Processing config can be inspected in a similar fashion as before

In [ ]:
print(yaml.dump(gsc.to_config()))

In [ ]:
obnb.label.LabelsetCollection

The `gsc` object is an instance of the `obnb.label.LabelsetCollection` object.
You can also convert it to a
[GMT](https://software.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats#GMT:_Gene_Matrix_Transposed_file_format_.28.2A.gmt.29)-like
dataframe by calling the `to_df` method.

The resulting dataframe is a table where the first three columns correspond to
the term ID, term info, and the number of genes associated with this term after
the processing. The rest of the columns are gene IDs that are associated with a
particular term, padded with `None`s.

In [ ]:
gsc.to_df()

## 3. Constructing dataset

### 3.1 The hard way: consolidate the network with gene set collection and combine into a dataset

- Pros: Flexible filtering and dataset construction to help investigate specific
  biological questions.
- Cons: Many steps involved to filter and pre-process.

In [ ]:
from obnb.label import filters
from obnb.label.split import RatioPartition
from obnb.util.converter import GenePropertyConverter


# Load PubMed count gene property converter
pubmedcnt_converter = GenePropertyConverter(root, name="PubMedCount")

# 6/2/2/ study-bias holdout split for genes
splitter = RatioPartition(0.6, 0.2, 0.2, ascending=False,
                          property_converter=pubmedcnt_converter)

# Apply filters to the gene set collection
gsc_filtered = gsc.apply(
    filters.Compose(
        # Only use genes that are present in the network
        filters.EntityExistenceFilter(list(g.node_ids), log_level="INFO",),
        # Remove any labelsets with less than 50 network genes
        filters.LabelsetRangeFilterSize(min_val=50, log_level="INFO",),
        # Make sure each split has at least 10 positive examples
        filters.LabelsetRangeFilterSplit(min_val=10, splitter=splitter, log_level="INFO",),
        log_level="INFO",
    ),
)

In [ ]:
# Combine into a OBNB dataset object
dataset = obnb.Dataset(
    graph=g,
    feature=g.to_dense_graph().to_feature(),
    label=gsc_filtered,
    splitter=splitter,
)

In [ ]:
dataset.graph

In [ ]:
dataset.label

### 3.2. The easy way: OBNB default dataset construction

- Pros: Easy to construct the dataset as it masked out a lot of common steps.
- Cons: Less flexible and hard to construct specialized datasets.

In [ ]:
dataset = obnb.OpenBiomedNetBench(
    root=root,
    graph_name="BioPlex",
    label_name="DisGeNET",
    version=data_version,
    graph_as_feature=True,
    use_dense_graph=True,
)

In [ ]:
# Similar to all previously shown cases, dataset have builtin cache utility
# to help spead up dataloading after the first instantiation.
dataset = obnb.OpenBiomedNetBench(
    root=root,
    graph_name="BioPlex",
    label_name="DisGeNET",
    version=data_version,
    graph_as_feature=True,
    use_dense_graph=True,
)

## 4. Simple model evaluation using the dataset and the builtin trianer

### 4.1. Label propagation

In [ ]:
import pandas as pd

from obnb.model_trainer import LabelPropagationTrainer
from obnb.model.label_propagation import OneHopPropagation

In [ ]:
lp_mdl = OneHopPropagation()
lp_trainer = LabelPropagationTrainer()

In [ ]:
lp_results = lp_trainer.fit_and_eval(lp_mdl, dataset)

In [ ]:
lp_df = pd.DataFrame(lp_results, index=dataset.label.label_ids)
lp_df

In [ ]:
lp_df.describe()

### 4.2. Supervised learning

In [ ]:
from sklearn.linear_model import LogisticRegression
from obnb.model_trainer import SupervisedLearningTrainer

In [ ]:
sl_mdl = LogisticRegression(penalty="l2", solver="lbfgs")
sl_trainer = SupervisedLearningTrainer()

In [ ]:
sl_results = sl_trainer.fit_and_eval(sl_mdl, dataset)

In [ ]:
sl_df = pd.DataFrame(sl_results, index=dataset.label.label_ids)
sl_df

In [ ]:
sl_df.describe()

### 4.3. GNN (coming soon)